# 🚀 Google Colab Setup

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ogautier1980/sandbox-ml/blob/main/cours/XX_CHAPTER/XX_NOTEBOOK.ipynb)

**Si vous exécutez ce notebook sur Google Colab**, exécutez la cellule suivante pour installer les dépendances.

In [ ]:
# Installation des dépendances (Google Colab uniquement)import sysIN_COLAB = 'google.colab' in sys.modulesif IN_COLAB:    print('📦 Installation des packages...')        # Packages ML de base    !pip install -q numpy pandas matplotlib seaborn scikit-learn        # Détection du chapitre et installation des dépendances spécifiques    notebook_name = '08_demo_rag_llm.ipynb'  # Sera remplacé automatiquement        # Ch 06-08 : Deep Learning    if any(x in notebook_name for x in ['06_', '07_', '08_']):        !pip install -q torch torchvision torchaudio        # Ch 08 : NLP    if '08_' in notebook_name:        !pip install -q transformers datasets tokenizers        if 'rag' in notebook_name:            !pip install -q sentence-transformers faiss-cpu rank-bm25        # Ch 09 : Reinforcement Learning    if '09_' in notebook_name:        !pip install -q gymnasium[classic-control]        # Ch 04 : Boosting    if '04_' in notebook_name and 'boosting' in notebook_name:        !pip install -q xgboost lightgbm catboost        # Ch 05 : Clustering avancé    if '05_' in notebook_name:        !pip install -q umap-learn        # Ch 11 : Séries temporelles    if '11_' in notebook_name:        !pip install -q statsmodels prophet        # Ch 12 : Vision avancée    if '12_' in notebook_name:        !pip install -q ultralytics timm segmentation-models-pytorch        # Ch 13 : Recommandation    if '13_' in notebook_name:        !pip install -q scikit-surprise implicit        # Ch 14 : MLOps    if '14_' in notebook_name:        !pip install -q mlflow fastapi pydantic        print('✅ Installation terminée !')else:    print('ℹ️  Environnement local détecté, les packages sont déjà installés.')

# Chapitre 08 - RAG et LLMs : Retrieval-Augmented Generation

Ce notebook explore les concepts avancés de **RAG (Retrieval-Augmented Generation)**, une technique qui combine la recherche d'information avec la génération de texte par LLMs.

## Objectifs
1. Comprendre l'architecture RAG
2. Implémenter des embeddings et vector search avec FAISS
3. Construire un pipeline RAG complet
4. Techniques avancées : chunking, reranking, hybrid search
5. Évaluer la qualité du RAG

## 1. Introduction au RAG

### Qu'est-ce que le RAG ?

Le **Retrieval-Augmented Generation** est une technique qui améliore les LLMs en leur fournissant des informations contextuelles pertinentes récupérées d'une base de connaissances.

**Architecture RAG** :
```
Query → Retrieval (recherche) → Context → LLM → Response
         ↓
    Knowledge Base
    (documents vectorisés)
```

**Avantages** :
- Réduit les hallucinations du LLM
- Permet d'utiliser des connaissances à jour sans réentraînement
- Transparence : les sources peuvent être citées

**Cas d'usage** :
- Chatbots de support client
- Systèmes de Q&A sur documentation
- Assistants de recherche académique

In [ ]:
# Installation des bibliothèques nécessaires
!pip install -q sentence-transformers faiss-cpu rank-bm25 scikit-learn matplotlib numpy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer, CrossEncoder
import faiss
from sklearn.decomposition import PCA
from rank_bm25 import BM25Okapi
import warnings
warnings.filterwarnings('ignore')

## 2. Embeddings et Vector Search

### 2.1 Création d'Embeddings avec Sentence-BERT

Les **embeddings** sont des représentations vectorielles denses qui capturent le sens sémantique du texte.

In [ ]:
# Charger un modèle d'embeddings pré-entraîné
# all-MiniLM-L6-v2 : léger (80 MB), rapide, 384 dimensions
model = SentenceTransformer('all-MiniLM-L6-v2')

print(f"Modèle chargé : {model.get_sentence_embedding_dimension()} dimensions")

In [ ]:
# Dataset exemple : corpus de connaissances ML
documents = [
    "Le Machine Learning est une branche de l'intelligence artificielle qui permet aux systèmes d'apprendre automatiquement à partir des données sans être explicitement programmés.",
    "Les réseaux de neurones profonds sont composés de plusieurs couches de neurones artificiels interconnectés qui transforment progressivement les données d'entrée.",
    "LSTM signifie Long Short-Term Memory. C'est une architecture de réseau de neurones récurrent conçue pour résoudre le problème du gradient qui disparaît.",
    "L'attention est un mécanisme qui permet au modèle de se concentrer sur différentes parties de l'entrée lors de la génération de chaque élément de sortie.",
    "Les Transformers utilisent uniquement des mécanismes d'attention et se passent complètement de récurrence, ce qui permet une meilleure parallélisation.",
    "BERT (Bidirectional Encoder Representations from Transformers) est pré-entraîné avec une tâche de Masked Language Modeling sur de grands corpus.",
    "GPT (Generative Pre-trained Transformer) est un modèle autorégressif entraîné à prédire le mot suivant dans une séquence.",
    "Le fine-tuning consiste à adapter un modèle pré-entraîné à une tâche spécifique en continuant l'entraînement sur des données de cette tâche.",
    "La validation croisée permet d'évaluer la performance d'un modèle en divisant les données en plusieurs folds et en entraînant/testant sur différentes combinaisons.",
    "L'overfitting se produit quand un modèle apprend trop bien les données d'entraînement au point de ne plus généraliser sur de nouvelles données.",
    "La régularisation L2 (Ridge) ajoute une pénalité proportionnelle au carré des poids pour éviter l'overfitting.",
    "Le dropout est une technique de régularisation qui désactive aléatoirement des neurones pendant l'entraînement pour forcer le réseau à être plus robuste.",
    "Le gradient descent est un algorithme d'optimisation qui ajuste itérativement les paramètres dans la direction opposée au gradient de la fonction de perte.",
    "Adam (Adaptive Moment Estimation) est un optimiseur qui combine les avantages de RMSProp et momentum avec des taux d'apprentissage adaptatifs.",
    "La backpropagation est l'algorithme utilisé pour calculer les gradients dans les réseaux de neurones en propageant l'erreur de la sortie vers l'entrée."
]

print(f"Corpus de {len(documents)} documents chargé")

In [ ]:
# Générer les embeddings pour tous les documents
print("Génération des embeddings...")
embeddings = model.encode(documents, show_progress_bar=True)

print(f"\nShape des embeddings : {embeddings.shape}")
print(f"Type : {embeddings.dtype}")
print(f"Exemple (5 premières dimensions du doc 0) : {embeddings[0][:5]}")

### 2.2 Vector Search avec FAISS

**FAISS** (Facebook AI Similarity Search) est une bibliothèque optimisée pour la recherche de similarité dans des espaces vectoriels de haute dimension.

In [ ]:
# Créer un index FAISS pour recherche rapide
dimension = embeddings.shape[1]  # 384 pour all-MiniLM-L6-v2

# IndexFlatL2 : recherche exhaustive avec distance L2 (Euclidienne)
index = faiss.IndexFlatL2(dimension)

# Ajouter les embeddings à l'index
index.add(embeddings.astype('float32'))

print(f"Index FAISS créé avec {index.ntotal} vecteurs")

In [ ]:
# Fonction de recherche
def search(query, k=3):
    """
    Recherche les k documents les plus similaires à la query.
    
    Args:
        query (str): Question/requête de l'utilisateur
        k (int): Nombre de résultats à retourner
    
    Returns:
        List[Tuple[str, float]]: Liste de (document, distance)
    """
    # Encoder la query
    query_embedding = model.encode([query])
    
    # Rechercher dans l'index
    distances, indices = index.search(query_embedding.astype('float32'), k)
    
    # Retourner les résultats
    results = []
    for j, i in enumerate(indices[0]):
        results.append((documents[i], float(distances[0][j])))
    
    return results

In [ ]:
# Test de recherche
query = "Comment fonctionnent les LSTM ?"
results = search(query, k=3)

print(f"Query : '{query}'\n")
print("Top 3 résultats :")
print("=" * 80)
for i, (doc, dist) in enumerate(results, 1):
    print(f"\n[{i}] Distance L2: {dist:.3f}")
    print(f"Document : {doc}")
    print("-" * 80)

In [ ]:
# Tester plusieurs queries
test_queries = [
    "Qu'est-ce que l'attention dans les réseaux de neurones ?",
    "Comment éviter l'overfitting ?",
    "Quelle est la différence entre BERT et GPT ?"
]

for query in test_queries:
    print(f"\n{'='*80}")
    print(f"Query : '{query}'")
    print(f"{'='*80}")
    results = search(query, k=2)
    for i, (doc, dist) in enumerate(results, 1):
        print(f"\n[{i}] Distance: {dist:.3f}")
        print(f"{doc[:150]}...")

## 3. Pipeline RAG Complet

Construisons un système RAG end-to-end qui intègre retrieval et génération de contexte.

In [ ]:
class SimpleRAG:
    """
    Pipeline RAG simple : Retrieval + Context Generation.
    
    Note : Cette implémentation ne fait pas appel à un LLM réel (OpenAI/GPT)
    mais génère un prompt formaté qui pourrait être envoyé à un LLM.
    """
    
    def __init__(self, documents, model_name='all-MiniLM-L6-v2'):
        self.documents = documents
        self.model = SentenceTransformer(model_name)
        self.embeddings = self.model.encode(documents, show_progress_bar=False)
        self.index = self._create_index()
    
    def _create_index(self):
        """Crée un index FAISS pour la recherche vectorielle."""
        dimension = self.embeddings.shape[1]
        index = faiss.IndexFlatL2(dimension)
        index.add(self.embeddings.astype('float32'))
        return index
    
    def retrieve(self, query, k=3):
        """
        Récupère les k documents les plus pertinents.
        
        Args:
            query (str): Question de l'utilisateur
            k (int): Nombre de documents à récupérer
        
        Returns:
            List[str]: Liste des documents pertinents
        """
        query_emb = self.model.encode([query])
        distances, indices = self.index.search(query_emb.astype('float32'), k)
        return [self.documents[i] for i in indices[0]]
    
    def generate_context(self, query, k=3):
        """
        Génère un contexte formaté à partir des documents récupérés.
        
        Args:
            query (str): Question de l'utilisateur
            k (int): Nombre de documents à utiliser
        
        Returns:
            str: Contexte formaté
        """
        retrieved_docs = self.retrieve(query, k=k)
        context = "\n\n".join([
            f"Document {i+1}: {doc}" 
            for i, doc in enumerate(retrieved_docs)
        ])
        return context
    
    def answer(self, query, k=3):
        """
        Génère un prompt complet pour un LLM avec contexte récupéré.
        
        Args:
            query (str): Question de l'utilisateur
            k (int): Nombre de documents à utiliser
        
        Returns:
            str: Prompt formaté pour LLM
        """
        context = self.generate_context(query, k=k)
        
        prompt = f"""Vous êtes un assistant IA expert en Machine Learning.

Contexte récupéré de la base de connaissances :
{context}

Question de l'utilisateur : {query}

Instructions :
- Répondez uniquement en vous basant sur le contexte fourni
- Si l'information n'est pas dans le contexte, dites-le clairement
- Citez les documents pertinents (Document 1, 2, etc.)

Réponse :"""
        
        return prompt

In [ ]:
# Instancier le système RAG
rag = SimpleRAG(documents)

print("Système RAG initialisé avec succès !")
print(f"Nombre de documents indexés : {len(documents)}")

In [ ]:
# Test du pipeline RAG
query = "Qu'est-ce qu'un LSTM et pourquoi est-il utilisé ?"
prompt = rag.answer(query, k=3)

print(prompt)

In [ ]:
# Tester avec différentes questions
test_queries = [
    "Comment fonctionne le mécanisme d'attention ?",
    "Quelle est la différence entre BERT et GPT ?",
    "Quelles sont les techniques pour éviter l'overfitting ?"
]

for query in test_queries:
    print(f"\n{'='*80}")
    print(f"QUERY : {query}")
    print(f"{'='*80}")
    
    # Récupérer seulement les documents (sans le prompt complet)
    retrieved = rag.retrieve(query, k=2)
    for i, doc in enumerate(retrieved, 1):
        print(f"\n[Document {i}]")
        print(doc)

## 4. Chunking et Preprocessing

Pour des documents longs, il faut les **découper en chunks** (morceaux) gérables.

In [ ]:
def chunk_text(text, chunk_size=200, overlap=50):
    """
    Découpe un texte en chunks avec overlap.
    
    Args:
        text (str): Texte à découper
        chunk_size (int): Nombre de mots par chunk
        overlap (int): Nombre de mots de chevauchement entre chunks
    
    Returns:
        List[str]: Liste de chunks
    """
    words = text.split()
    chunks = []
    
    for i in range(0, len(words), chunk_size - overlap):
        chunk = ' '.join(words[i:i + chunk_size])
        if chunk:  # Éviter les chunks vides
            chunks.append(chunk)
    
    return chunks

In [ ]:
# Exemple avec un document long (simulation)
long_document = """Le Machine Learning est une discipline passionnante. 
Elle permet de créer des systèmes intelligents. Les algorithmes apprennent des données. 
Il existe plusieurs types d'apprentissage : supervisé, non-supervisé et par renforcement. 
Les réseaux de neurones sont des modèles très puissants. Ils sont inspirés du cerveau humain.
""" * 20  # Répéter pour simuler un long document

print(f"Document original : {len(long_document.split())} mots\n")

# Découper avec différents paramètres
chunks = chunk_text(long_document, chunk_size=50, overlap=10)

print(f"Document découpé en {len(chunks)} chunks")
print(f"\nExemple de chunks :")
for i, chunk in enumerate(chunks[:3], 1):
    print(f"\n[Chunk {i}] ({len(chunk.split())} mots)")
    print(chunk)

In [ ]:
def semantic_chunk(text, separator='\n\n'):
    """
    Découpe un texte par séparateur sémantique (paragraphes).
    
    Args:
        text (str): Texte à découper
        separator (str): Séparateur (défaut : double saut de ligne)
    
    Returns:
        List[str]: Liste de chunks
    """
    paragraphs = text.split(separator)
    return [p.strip() for p in paragraphs if p.strip()]

# Exemple
text_with_paragraphs = """Paragraphe 1 : Introduction au ML.
Le Machine Learning transforme les données en connaissances.

Paragraphe 2 : Les algorithmes.
Il existe de nombreux algorithmes différents.

Paragraphe 3 : Applications.
Le ML est utilisé dans de nombreux domaines."""

semantic_chunks = semantic_chunk(text_with_paragraphs)
print(f"Chunks sémantiques : {len(semantic_chunks)}\n")
for i, chunk in enumerate(semantic_chunks, 1):
    print(f"[Chunk {i}]\n{chunk}\n")

## 5. Évaluation du RAG

Mesurer la qualité du retrieval avec **Precision@K** et **Recall@K**.

In [ ]:
def evaluate_retrieval(queries, expected_indices, rag_system, k=3):
    """
    Évalue la qualité du retrieval avec Precision@K et Recall@K.
    
    Args:
        queries (List[str]): Liste de questions
        expected_indices (List[List[int]]): Indices des documents pertinents pour chaque query
        rag_system (SimpleRAG): Système RAG à évaluer
        k (int): Nombre de documents à récupérer
    
    Returns:
        Tuple[float, float]: (mean_precision, mean_recall)
    """
    precisions = []
    recalls = []
    
    for query, expected in zip(queries, expected_indices):
        # Récupérer les documents
        retrieved_docs = rag_system.retrieve(query, k=k)
        
        # Trouver les indices des documents récupérés
        retrieved_indices = []
        for doc in retrieved_docs:
            try:
                idx = rag_system.documents.index(doc)
                retrieved_indices.append(idx)
            except ValueError:
                pass
        
        # Calculer Precision et Recall
        retrieved_set = set(retrieved_indices)
        expected_set = set(expected)
        
        true_positives = len(retrieved_set & expected_set)
        
        precision = true_positives / k if k > 0 else 0
        recall = true_positives / len(expected_set) if expected_set else 0
        
        precisions.append(precision)
        recalls.append(recall)
    
    return np.mean(precisions), np.mean(recalls)

In [ ]:
# Dataset d'évaluation
eval_queries = [
    "Qu'est-ce que LSTM ?",
    "Comment éviter l'overfitting ?",
    "Qu'est-ce que l'attention ?"
]

# Indices des documents pertinents (ground truth)
# Basé sur notre corpus de 15 documents
expected_indices = [
    [2],        # Query 1 : doc sur LSTM (index 2)
    [9, 11],    # Query 2 : docs sur overfitting et régularisation (index 9, 11)
    [3, 4]      # Query 3 : docs sur attention (index 3, 4)
]

# Évaluation
precision, recall = evaluate_retrieval(eval_queries, expected_indices, rag, k=3)

print(f"Precision@3 : {precision:.3f}")
print(f"Recall@3    : {recall:.3f}")

In [ ]:
# Évaluer pour différentes valeurs de K
k_values = [1, 2, 3, 5]
results = []

for k in k_values:
    prec, rec = evaluate_retrieval(eval_queries, expected_indices, rag, k=k)
    results.append((k, prec, rec))
    print(f"K={k} | Precision: {prec:.3f} | Recall: {rec:.3f}")

# Visualisation
k_vals, precs, recs = zip(*results)

plt.figure(figsize=(10, 5))
plt.plot(k_vals, precs, marker='o', label='Precision@K', linewidth=2)
plt.plot(k_vals, recs, marker='s', label='Recall@K', linewidth=2)
plt.xlabel('K (nombre de documents récupérés)', fontsize=12)
plt.ylabel('Score', fontsize=12)
plt.title('Precision@K et Recall@K en fonction de K', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.xticks(k_vals)
plt.tight_layout()
plt.show()

## 6. Techniques Avancées

### 6.1 Reranking : Améliorer les Résultats

Le **reranking** utilise un modèle plus précis (CrossEncoder) pour réordonner les résultats de la première recherche.

In [ ]:
# Charger un modèle de reranking
print("Chargement du modèle de reranking...")
reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
print("Modèle chargé !")

In [ ]:
def rerank(query, documents, top_k=3):
    """
    Réordonne les documents avec un CrossEncoder.
    
    Args:
        query (str): Question de l'utilisateur
        documents (List[str]): Documents à réordonner
        top_k (int): Nombre de résultats à retourner
    
    Returns:
        List[Tuple[str, float]]: Documents réordonnés avec scores
    """
    # Créer des paires (query, document)
    pairs = [[query, doc] for doc in documents]
    
    # Prédire les scores de pertinence
    scores = reranker.predict(pairs)
    
    # Réordonner par score décroissant
    ranked_indices = np.argsort(scores)[::-1][:top_k]
    
    return [(documents[i], float(scores[i])) for i in ranked_indices]

In [ ]:
# Comparaison : Retrieval seul vs Retrieval + Reranking
query = "Expliquez le mécanisme d'attention dans les Transformers"

# 1. Retrieval classique (top 5)
retrieved_docs = rag.retrieve(query, k=5)

print("RETRIEVAL SEUL (FAISS) - Top 5")
print("="*80)
for i, doc in enumerate(retrieved_docs, 1):
    print(f"[{i}] {doc[:100]}...\n")

# 2. Reranking des résultats
reranked = rerank(query, retrieved_docs, top_k=3)

print("\nAPRÈS RERANKING - Top 3")
print("="*80)
for i, (doc, score) in enumerate(reranked, 1):
    print(f"[{i}] Score: {score:.3f}")
    print(f"{doc[:100]}...\n")

### 6.2 Hybrid Search : BM25 + Dense Embeddings

Combine **recherche par mots-clés (BM25)** et **recherche sémantique (embeddings)**.

In [ ]:
def hybrid_search(query, documents, embeddings, index, model, alpha=0.5, k=3):
    """
    Recherche hybride : BM25 + Dense embeddings.
    
    Args:
        query (str): Question de l'utilisateur
        documents (List[str]): Corpus de documents
        embeddings (np.ndarray): Embeddings des documents
        index (faiss.Index): Index FAISS
        model (SentenceTransformer): Modèle d'embeddings
        alpha (float): Poids de BM25 (0=dense only, 1=BM25 only)
        k (int): Nombre de résultats
    
    Returns:
        List[Tuple[str, float]]: Documents avec scores hybrides
    """
    # 1. BM25 (keyword-based)
    tokenized_docs = [doc.lower().split() for doc in documents]
    bm25 = BM25Okapi(tokenized_docs)
    bm25_scores = bm25.get_scores(query.lower().split())
    
    # Normaliser scores BM25 (0-1)
    if bm25_scores.max() > 0:
        bm25_scores = bm25_scores / bm25_scores.max()
    
    # 2. Dense search (embedding-based)
    query_emb = model.encode([query])
    distances, indices = index.search(query_emb.astype('float32'), len(documents))
    
    # Convertir distances L2 en scores de similarité (inverse)
    dense_scores = np.zeros(len(documents))
    max_dist = distances[0].max() if distances[0].max() > 0 else 1
    for i, idx in enumerate(indices[0]):
        dense_scores[idx] = 1 - (distances[0][i] / max_dist)
    
    # 3. Combiner les scores
    combined_scores = alpha * bm25_scores + (1 - alpha) * dense_scores
    
    # 4. Sélectionner top-k
    top_indices = np.argsort(combined_scores)[::-1][:k]
    
    return [(documents[i], float(combined_scores[i])) for i in top_indices]

In [ ]:
# Comparaison des stratégies de recherche
query = "gradient descent optimisation"

print(f"Query : '{query}'\n")
print("="*80)

# BM25 only (alpha=1)
bm25_results = hybrid_search(query, documents, embeddings, index, model, alpha=1.0, k=3)
print("\nBM25 ONLY (Keyword-based)")
for i, (doc, score) in enumerate(bm25_results, 1):
    print(f"[{i}] Score: {score:.3f} | {doc[:80]}...")

# Dense only (alpha=0)
dense_results = hybrid_search(query, documents, embeddings, index, model, alpha=0.0, k=3)
print("\nDENSE ONLY (Semantic)")
for i, (doc, score) in enumerate(dense_results, 1):
    print(f"[{i}] Score: {score:.3f} | {doc[:80]}...")

# Hybrid (alpha=0.5)
hybrid_results = hybrid_search(query, documents, embeddings, index, model, alpha=0.5, k=3)
print("\nHYBRID (BM25 + Dense, alpha=0.5)")
for i, (doc, score) in enumerate(hybrid_results, 1):
    print(f"[{i}] Score: {score:.3f} | {doc[:80]}...")

## 7. Visualisation des Embeddings

Visualiser les embeddings en 2D avec **PCA**.

In [ ]:
# Réduction de dimensionnalité avec PCA
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(embeddings)

print(f"Variance expliquée par les 2 composantes : {pca.explained_variance_ratio_.sum():.2%}")

In [ ]:
# Visualisation
plt.figure(figsize=(14, 10))
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], s=100, alpha=0.6, c='steelblue', edgecolors='black')

# Annoter avec les premiers mots de chaque document
for i, doc in enumerate(documents):
    label = ' '.join(doc.split()[:4]) + '...'  # 4 premiers mots
    plt.annotate(label, 
                 (embeddings_2d[i, 0], embeddings_2d[i, 1]),
                 fontsize=8,
                 alpha=0.8,
                 bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.3))

plt.title('Visualisation des Embeddings (PCA 2D)', fontsize=14, fontweight='bold')
plt.xlabel('Composante Principale 1', fontsize=12)
plt.ylabel('Composante Principale 2', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Visualiser une query et ses résultats
query_test = "Qu'est-ce que le dropout ?"
query_emb = model.encode([query_test])
query_2d = pca.transform(query_emb)

# Récupérer les documents pertinents
retrieved_indices = []
retrieved_docs = rag.retrieve(query_test, k=3)
for doc in retrieved_docs:
    retrieved_indices.append(documents.index(doc))

# Visualisation
plt.figure(figsize=(14, 10))

# Documents (gris)
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], s=100, alpha=0.3, c='gray', label='Autres documents')

# Documents récupérés (vert)
retrieved_2d = embeddings_2d[retrieved_indices]
plt.scatter(retrieved_2d[:, 0], retrieved_2d[:, 1], s=150, alpha=0.8, c='green', edgecolors='black', label='Docs récupérés')

# Query (rouge)
plt.scatter(query_2d[:, 0], query_2d[:, 1], s=200, c='red', marker='*', edgecolors='black', label='Query', zorder=5)

# Annotations
plt.annotate('QUERY', (query_2d[0, 0], query_2d[0, 1]), fontsize=12, fontweight='bold', color='red')
for i in retrieved_indices:
    label = ' '.join(documents[i].split()[:4]) + '...'
    plt.annotate(label, (embeddings_2d[i, 0], embeddings_2d[i, 1]), fontsize=9, color='green')

plt.title(f'Query : "{query_test}"', fontsize=14, fontweight='bold')
plt.xlabel('Composante Principale 1', fontsize=12)
plt.ylabel('Composante Principale 2', fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 8. Exercices Pratiques

### Exercice 1 : Créer un RAG pour un Corpus Personnalisé

**Tâche** : Créez un système RAG sur un corpus de votre choix (ex: articles scientifiques, documentation technique, etc.).

**Instructions** :
1. Préparez un corpus d'au moins 20 documents
2. Implémentez le chunking si nécessaire
3. Créez l'index FAISS
4. Testez avec 5 queries
5. Évaluez avec Precision@3 et Recall@3

In [ ]:
# SOLUTION EXERCICE 1

# Corpus exemple : concepts Python avancés
python_docs = [
    "Les décorateurs en Python sont des fonctions qui modifient le comportement d'autres fonctions ou classes.",
    "Les générateurs permettent de créer des itérateurs de manière élégante en utilisant le mot-clé yield.",
    "Le context manager (with statement) gère automatiquement l'acquisition et la libération de ressources.",
    "Les list comprehensions offrent une syntaxe concise pour créer des listes à partir d'itérables.",
    "Les lambda functions sont des fonctions anonymes définies en une seule ligne avec le mot-clé lambda.",
    "Le Global Interpreter Lock (GIL) empêche plusieurs threads d'exécuter du bytecode Python simultanément.",
    "Les metaclasses sont des classes de classes qui contrôlent la création et le comportement des classes.",
    "Le duck typing permet d'utiliser des objets basés sur leurs méthodes plutôt que sur leur type explicite.",
    "Les asyncio et await permettent la programmation asynchrone pour gérer les opérations I/O efficacement.",
    "Le property decorator transforme une méthode en attribut accessible avec une syntaxe simplifiée.",
    "Les dataclasses (depuis Python 3.7) réduisent le boilerplate pour créer des classes de données.",
    "Le module collections fournit des conteneurs spécialisés comme defaultdict, Counter et deque.",
    "Les type hints (PEP 484) permettent d'annoter le code avec des types pour améliorer la lisibilité.",
    "Le pattern matching (Python 3.10+) permet de comparer des structures de données avec match/case.",
    "Les f-strings offrent une syntaxe moderne et performante pour le formatage de chaînes.",
    "Le module functools fournit des outils pour la programmation fonctionnelle comme partial et reduce.",
    "Les slots optimisent la mémoire en définissant explicitement les attributs d'une classe.",
    "Le module abc (Abstract Base Classes) permet de définir des interfaces en Python.",
    "Les coroutines sont des fonctions qui peuvent être suspendues et reprises, essentielles pour asyncio.",
    "Le module itertools fournit des itérateurs efficaces pour des opérations combinatoires et de permutation."
]

# Créer le système RAG
python_rag = SimpleRAG(python_docs)

# Test avec queries
test_queries = [
    "Comment créer des itérateurs en Python ?",
    "Qu'est-ce que le GIL ?",
    "Comment formater des chaînes en Python ?",
    "Qu'est-ce qu'une fonction lambda ?",
    "Comment faire de la programmation asynchrone ?"
]

print("Test du RAG sur documentation Python\n")
for query in test_queries:
    print(f"Query : {query}")
    results = python_rag.retrieve(query, k=2)
    for i, doc in enumerate(results, 1):
        print(f"  [{i}] {doc}")
    print()

### Exercice 2 : Comparer Différents Modèles d'Embeddings

**Tâche** : Comparez les performances de différents modèles d'embeddings.

**Modèles à tester** :
- `all-MiniLM-L6-v2` (rapide, 384 dim)
- `all-mpnet-base-v2` (meilleur qualité, 768 dim)

**Métrique** : Precision@3 sur un ensemble de queries

In [ ]:
# SOLUTION EXERCICE 2

import time

models_to_test = [
    'all-MiniLM-L6-v2',      # 384 dimensions
    'all-mpnet-base-v2'      # 768 dimensions
]

# Queries et ground truth (basé sur notre corpus ML original)
eval_queries = [
    "Qu'est-ce que LSTM ?",
    "Comment éviter l'overfitting ?",
    "Qu'est-ce que l'attention ?"
]
expected_indices = [[2], [9, 11], [3, 4]]

comparison_results = []

for model_name in models_to_test:
    print(f"\nTest du modèle : {model_name}")
    print("="*60)
    
    # Créer RAG avec ce modèle
    start = time.time()
    rag_test = SimpleRAG(documents, model_name=model_name)
    init_time = time.time() - start
    
    # Évaluer
    prec, rec = evaluate_retrieval(eval_queries, expected_indices, rag_test, k=3)
    
    comparison_results.append({
        'model': model_name,
        'precision': prec,
        'recall': rec,
        'init_time': init_time,
        'dimensions': rag_test.embeddings.shape[1]
    })
    
    print(f"Dimensions      : {rag_test.embeddings.shape[1]}")
    print(f"Temps init      : {init_time:.2f}s")
    print(f"Precision@3     : {prec:.3f}")
    print(f"Recall@3        : {rec:.3f}")

# Tableau comparatif
print("\n" + "="*80)
print("COMPARAISON DES MODÈLES")
print("="*80)
print(f"{'Modèle':<30} {'Dim':<8} {'P@3':<8} {'R@3':<8} {'Temps (s)':<10}")
print("-"*80)
for res in comparison_results:
    print(f"{res['model']:<30} {res['dimensions']:<8} {res['precision']:<8.3f} {res['recall']:<8.3f} {res['init_time']:<10.2f}")

### Exercice 3 : Implémenter un Système de Reranking

**Tâche** : Implémentez un pipeline complet avec retrieval initial (k=10) puis reranking (top-3).

**Étapes** :
1. Récupérer 10 documents avec FAISS
2. Reranker avec CrossEncoder
3. Garder les 3 meilleurs
4. Comparer avec retrieval direct (k=3)

In [ ]:
# SOLUTION EXERCICE 3

class RAGWithReranking:
    """
    Pipeline RAG avec étape de reranking.
    """
    
    def __init__(self, documents, embedding_model='all-MiniLM-L6-v2', 
                 reranker_model='cross-encoder/ms-marco-MiniLM-L-6-v2'):
        self.documents = documents
        self.embedding_model = SentenceTransformer(embedding_model)
        self.reranker = CrossEncoder(reranker_model)
        self.embeddings = self.embedding_model.encode(documents, show_progress_bar=False)
        self.index = self._create_index()
    
    def _create_index(self):
        dimension = self.embeddings.shape[1]
        index = faiss.IndexFlatL2(dimension)
        index.add(self.embeddings.astype('float32'))
        return index
    
    def retrieve_and_rerank(self, query, retrieve_k=10, final_k=3):
        """
        Pipeline : Retrieval (k=retrieve_k) → Reranking → Top final_k.
        """
        # Étape 1 : Retrieval initial
        query_emb = self.embedding_model.encode([query])
        distances, indices = self.index.search(query_emb.astype('float32'), retrieve_k)
        candidate_docs = [self.documents[i] for i in indices[0]]
        
        # Étape 2 : Reranking
        pairs = [[query, doc] for doc in candidate_docs]
        scores = self.reranker.predict(pairs)
        
        # Étape 3 : Top final_k
        ranked_indices = np.argsort(scores)[::-1][:final_k]
        final_docs = [(candidate_docs[i], float(scores[i])) for i in ranked_indices]
        
        return final_docs

# Créer le système
print("Initialisation du système avec reranking...")
rag_rerank = RAGWithReranking(documents)
print("Système prêt !\n")

# Test
query = "Expliquez la backpropagation dans les réseaux de neurones"

print(f"Query : '{query}'\n")
print("="*80)

# Résultats avec reranking
results_rerank = rag_rerank.retrieve_and_rerank(query, retrieve_k=10, final_k=3)
print("AVEC RERANKING (retrieve 10 → rerank → top 3)")
for i, (doc, score) in enumerate(results_rerank, 1):
    print(f"\n[{i}] Score: {score:.3f}")
    print(f"{doc}")

# Comparaison avec retrieval direct
print("\n" + "="*80)
results_direct = rag.retrieve(query, k=3)
print("SANS RERANKING (retrieval direct top 3)")
for i, doc in enumerate(results_direct, 1):
    print(f"\n[{i}] {doc}")

## Conclusion

### Récapitulatif

Dans ce notebook, nous avons exploré :

1. **Architecture RAG** : Retrieval + Generation pour améliorer les LLMs
2. **Embeddings** : Représentations vectorielles avec Sentence-BERT
3. **Vector Search** : Recherche rapide avec FAISS (IndexFlatL2)
4. **Pipeline RAG** : Système complet de retrieval et génération de contexte
5. **Chunking** : Techniques de découpage de documents longs
6. **Évaluation** : Precision@K et Recall@K pour mesurer la qualité
7. **Techniques avancées** :
   - **Reranking** : CrossEncoder pour améliorer les résultats
   - **Hybrid Search** : Combinaison BM25 (keyword) + Dense (semantic)
8. **Visualisation** : PCA pour explorer l'espace des embeddings

### Points Clés

- **RAG réduit les hallucinations** en fournissant un contexte factuel au LLM
- **FAISS est essentiel** pour la recherche vectorielle à grande échelle
- **Reranking améliore significativement** la qualité des résultats (mais coût computationnel)
- **Hybrid search combine** les avantages de la recherche par mots-clés et sémantique
- **L'évaluation est cruciale** : toujours mesurer Precision/Recall

### Pour Aller Plus Loin

- **LlamaIndex** : framework complet pour RAG
- **LangChain** : orchestration de LLMs avec RAG
- **ChromaDB, Pinecone, Weaviate** : bases de données vectorielles
- **ColBERT** : late interaction pour retrieval ultra-performant
- **Dense Passage Retrieval (DPR)** : modèle bi-encoder entraîné end-to-end